In [ ]:
from fastlane_bot.tests.deterministic.base_imports import *
from fastlane_bot.data.abi import CARBON_CONTROLLER_ABI, UNISWAP_V3_POOL_ABI, UNISWAP_V2_POOL_ABI, PANCAKESWAP_V3_POOL_ABI, PANCAKESWAP_V2_POOL_ABI
import time

# Select a new fork or existing

In [ ]:
## to add a click option for this
# url, fromBlock = create_new_testnet()
url, fromBlock = ('https://virtual.mainnet.rpc.tenderly.co/fb866397-29bd-4886-8406-a2cc7b7c5b1f',19117932) # mainnet example
url, fromBlock

In [ ]:
# Connect to the RPC:
w3 = Web3(Web3.HTTPProvider(url))
assert w3.is_connected() == True, "Web3 not connected"

In [ ]:
# Initialize the Carbon Controller contract
CarbonController_address = pd.read_csv(r"fastlane_bot\data\multichain_addresses.csv").query("exchange_name=='carbon_v1'").query("chain=='ethereum'").factory_address.values[0]
CarbonController = w3.eth.contract(address=CarbonController_address, abi=CARBON_CONTROLLER_ABI)

# Get Carbon Strategies and Delete

In [ ]:
# Get the current state
strategy_created_df, strategy_deleted_df, remaining_carbon_strategies = get_state_of_carbon_strategies(w3, CarbonController, fromBlock=1000000) #arbitrarily low blocknumber

# takes about 4 minutes per 100 strategies
# so 450 ~ 18 minutes
undeleted_strategies = delete_all_carbon_strategies(w3, CarbonController, carbon_strategy_id_owner_list=remaining_carbon_strategies)

# These strategies cannot be deleted on Ethereum
known_unable_to_delete = {
    68737038118029569619601670701217178714718: ("pDFS", "ETH"), #pDFS 
    }
assert all([x in known_unable_to_delete for x in undeleted_strategies]), f"Strategies not deleted that are unknown: {undeleted_strategies}"

# Redundant for checking state
strategy_created_df, strategy_deleted_df, remaining_carbon_strategies = get_state_of_carbon_strategies(w3, CarbonController, fromBlock=1000000) #arbitrarily low blocknumber

# The above resets the testnet

# Set the Test State on external pools
Since this method uses backdate_pool=True to fetch the current state of the external pools, then ALL the external pools included in a single test bot run must have their states updated prior to initializing the bot

In [ ]:
# Import pool data
static_pool_data_testing_path = os.path.normpath(r"fastlane_bot\data\blockchain_data\ethereum\static_pool_data_testing.csv")
test_pools = pd.read_csv(static_pool_data_testing_path, dtype=str)

# Handle each exchange_type differently for the required updates
for index in test_pools.index:
    exchange_type = test_pools["exchange_type"][index]
    print(test_pools["exchange_type"][index])
    handle_exchange_parameters(w3, test_pools.iloc[index])

In [ ]:
# Optional validation to ensure that all states are set correctly
# Calls all relevant functions and verifies slot content againt input data
run__slot_update_tests(w3, test_pools)

# Start a bot instance to run all tests against
- Limitations are one bot per mode per test

In [ ]:
# Start the bot
child_process = initialize_bot(
    blockchain = 'ethereum',
    arb_mode = 'multi',
    static_pool_data_FORTESTING = 'static_pool_data_testing', # this is the focused subset of pool data specfic for testing
    python_instance = sys.executable,
    rpc = url
    )

# Wait until the pool data populates
most_recent_log_folder = await_first_iteration()

In [ ]:
# Impore test strategies
test_strategies_path = os.path.normpath(r"fastlane_bot\data\blockchain_data\ethereum\test_strategies.json")
with open(test_strategies_path) as file:
    test_strategies = json.load(file)['test_strategies']
    print(f"{len(test_strategies.keys())} test strategies imported")

# Mark the block that new strats were created
strats_created_fromBlock = w3.eth.get_block_number()
print("strats_created_fromBlock", strats_created_fromBlock)

# populate a dictionary with all the relevant test strategies
test_strategy_txhashs = {}
for i in range(1,len(test_strategies.keys())+1):
    i = str(i)
    test_strategy = test_strategies[i]
    get_token_approval(w3, test_strategy['token0'], CarbonController.address, test_strategy['wallet'])
    get_token_approval(w3, test_strategy['token1'], CarbonController.address, test_strategy['wallet'])
    txhash = createStrategy_fromTestDict(w3, CarbonController, test_strategy)
    test_strategy_txhashs[i] = {}
    test_strategy_txhashs[i]['txhash'] = txhash

In [ ]:
# collect the new state of relevant strategies created
strategy_created_df, strategy_deleted_df, remaining_carbon_strategies = get_state_of_carbon_strategies(w3, CarbonController, fromBlock=strats_created_fromBlock) #arbitrarily low blocknumber
new_strats_created = strategy_created_df["id"].to_list()
print(f"There have been {len(new_strats_created)} new strategies created")

# add the strategy ids
for i in test_strategy_txhashs.keys():
    test_strategy_txhashs[i]['strategyid'] = strategy_created_df.query(f"transactionHash == '{test_strategy_txhashs[i]['txhash']}'").id.values[0]

# Wait for the arbs to execute
#this could be replaced with a loop searching for a successful tx
print("Sleeping while arbs are executed...")
time.sleep(int(35*len(test_strategy_txhashs.keys()) + 15)) 

# First layer verification
# Check that all strategies recently created have been traded on
updated_events_df = get_GenericEvents(w3, CarbonController, "StrategyUpdated", fromBlock=strats_created_fromBlock)
if len(updated_events_df) == 0:
    print("Not all strategies created have been traded against")
else:
    strategies_traded_against =  list(set(updated_events_df[updated_events_df.reason==1].id.to_list()))
if not all(item in strategies_traded_against for item in new_strats_created):
    print("Not all strategies created have been traded against")

In [ ]:
# get all successful_txs
all_successful_txs = glob.glob(os.path.join(most_recent_log_folder, "*.txt"))

# Read the succussful_txs in as strings
txt_all_successful_txs = []
for successful_tx in all_successful_txs:
    with open(successful_tx, 'r') as file:
        j = file.read()
        txt_all_successful_txs += [(j)]
        file.close()

# Successful transactions on Tenderly are marked by status=1
actually_txt_all_successful_txs = [tx for tx in txt_all_successful_txs if "'status': 1" in tx]

In [ ]:
# Import expected test results
test_results_path = os.path.normpath(r"fastlane_bot\data\blockchain_data\ethereum\test_results.json")
with open(test_results_path) as f:
    test_datas = json.load(f)['test_data']
    f.close()
    print(f"{len(test_datas.keys())} test results imported")

# Loop over the created test strategies and verify test data
for i in test_strategy_txhashs.keys():
    search_id = test_strategy_txhashs[i]['strategyid']
    print(f"Evaluating test {i}, {search_id}")
    tx_data = get_tx_data(search_id, actually_txt_all_successful_txs)
    clean_tx_data(tx_data)
    test_data = test_datas[i]
    if tx_data == test_data:
        print(f"Test {i} PASSED")
    else:
        print(f"Test {i} FAILED")
print("ALL TESTS PASSED")

In [ ]:
# Terminate the bot background process
if child_process.terminate():
    print("Arb Bot shutdown complete")